# EDA AAPI PROJECT

#Ethnic Group Listing

Decennial Census
T03004HOUSEHOLD TYPE (8 CATEGORIES)
select for Asain and PI

https://data.census.gov/

In [7]:
import pandas as pd

In [9]:
import pandas as pd

df = pd.read_csv('DECENNIALDDHCB2020.T03004-Data.csv', header=1)

# ethnicities are before" alone" using regex
df['Ethnic Group'] = df['Population Groups'].str.extract(r'(.*?) alone')

# replace 'Chinese, except Taiwanese' with 'Chinese'
df['Ethnic Group'] = df['Ethnic Group'].replace('Chinese, except Taiwanese', 'Chinese')

# drop "other"ʻs
df = df[~df['Ethnic Group'].str.contains('Other', na=False)]



display(df.head())

,Geography,Geographic Area Name,!!Total:,Race/Ethnic Group,Population Groups,!!Total:!!Family households:,!!Total:!!Family households:!!Married couple family,!!Total:!!Family households:!!Other family:,"!!Total:!!Family households:!!Other family:!!Male householder, no spouse present","!!Total:!!Family households:!!Other family:!!Female householder, no spouse present",!!Total:!!Nonfamily households:,!!Total:!!Nonfamily households:!!Householder living alone,!!Total:!!Nonfamily households:!!Householder not living alone,Unnamed: 13,Ethnic Group
0,0100000US,United States,2610671,3764,East Asian alone,1727918,1348018,379900,128855,251045,882753,657177,225576,NaN,East Asian
1,0100000US,United States,1493161,3765,"Chinese, except Taiwanese alone",999657,785490,214167,75503,138664,493504,346740,146764,NaN,Chinese
2,0100000US,United States,69679,3766,Hmong alone,58823,33607,25216,10848,14368,10856,6627,4229,NaN,Hmong
3,0100000US,United States,325588,3767,Japanese alone,191133,145523,45610,13109,32501,134455,115163,19292,NaN,Japanese
4,0100000US,United States,578144,3768,Korean alone,384584,307377,77207,23784,53423,193560,153453,40107,NaN,Korean


In [10]:
ethnic_group_list = sorted(df['Ethnic Group'].unique().tolist())
ethnic_group_list += ['AAPI', 'Asian American', 'Asian', 'Pacific Islander'] # adding in our general terms

print(ethnic_group_list)

['Afghan', 'Asian Indian', 'Bangladeshi', 'Bhutanese', 'Burmese', 'Cambodian', 'Central Asian', 'Chamorro', 'Chinese', 'Chuukese', 'East Asian', 'Fijian', 'Filipino', 'Guamanian', 'Hmong', 'Indonesian', 'Japanese', 'Kazakh', 'Korean', 'Kyrgyz', 'Laotian', 'Malaysian', 'Maori', 'Marshallese', 'Micronesian', 'Mien', 'Mongolian', 'Native Hawaiian', 'Nepalese', 'Pakistani', 'Palauan', 'Polynesian', 'Samoan', 'Sikh', 'Singaporean', 'South Asian', 'Southeast Asian', 'Sri Lankan', 'Tahitian', 'Taiwanese', 'Tajik', 'Thai', 'Tongan', 'Uzbek', 'Vietnamese', 'AAPI', 'Asian American', 'Asian', 'Pacific Islander']


In [14]:
ethnic_group_list = ['Afghan', 'Indian', 'Bangladeshi', 'Bhutanese', 'Burmese', 'Cambodian', 'Central Asian', 'Chamorro', 'Chinese', 'Chuukese', 'East Asian', 'Fijian', 'Filipino', 'Guamanian', 'Hmong', 'Indonesian', 'Japanese', 'Kazakh', 'Korean', 'Kyrgyz', 'Laotian', 'Malaysian', 'Maori', 'Marshallese', 'Micronesian', 'Mien', 'Mongolian', 'Native Hawaiian', 'Nepalese', 'Pakistani', 'Palauan', 'Polynesian', 'Samoan', 'Sikh', 'Singaporean', 'South Asian', 'Southeast Asian', 'Sri Lankan', 'Tahitian', 'Taiwanese', 'Tajik', 'Thai', 'Tongan', 'Uzbek', 'Vietnamese', 'AAPI', 'Asian American', 'Asian', 'Pacific Islander']

# Loading in Subset Common Crawl

using [common crawl october 2025](https://commoncrawl.org/blog/common-crawl-foundation-at-colm-2025)

3 things:
1. load figure out stuff
2. filter non english sites
3. keep file if it contains any of the ethniciies in the list

Kaiona's Current debate:
1.
- load the data into duck db (does gz work??)
- then wetfile for data
- alternatively can i used duck db for filtering in file and keyword search
- ie if "asian" is in the text file


2.
- now comprise a db with only "aapi" terms
- has to also be in english (preferably)
- one column sentance, one column term, one column url???

3.
- use pos tagging for all asian occurences
4. Data/modeling
- create a list of terms in relation from the pos tagging
- adjectives
- verbs
5.  Modeling (Pauses)
- use word2vec to train vectors for each word... ask kent at this point

data -> duck -> beautiful soup -> asian

at the bare minimum, i want to add onto the file format a column for asian type and asian

In [11]:
%pip install warcio
import requests
from warcio.archiveiterator import ArchiveIterator
import io

In [12]:
import requests
import gzip
import io

base = "https://data.commoncrawl.org/"
path = "crawl-data/CC-MAIN-2025-43/wet.paths.gz"
wet_paths_url = base + path

print(f"Downloading list of WET paths from: {wet_paths_url}")

try:
    response = requests.get(wet_paths_url, stream=True)
    response.raise_for_status() # Raise an HTTPError for bad responses

    # Decompress the gzipped content and read the paths
    with gzip.open(io.BytesIO(response.content), 'rt', encoding='utf-8') as f:
        wet_paths = [line.strip() for line in f if line.strip()]

    print(f"Successfully read {len(wet_paths)} WET file paths.")

    # Example: Get the URL for the first WET file
    if wet_paths:
        first_wet_path = wet_paths[0]
        first_wet_url = base + first_wet_path
        print(f"\nExample URL for the first WET file: {first_wet_url}")

        # You can now use first_wet_url to download and process the content of a WET file
        # (This part is not implemented in this cell, but shows the next step)
    else:
        print("The wet.paths.gz file was empty.")

except requests.exceptions.RequestException as e:
    print(f"Error downloading or reading wet.paths.gz: {e}")

Successfully read 100000 WET file paths.

Example URL for the first WET file: https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-43/segments/1759648357851.76/wet/CC-MAIN-20251005114239-20251005144239-00000.warc.wet.gz


Common Crawl WET (Web Extracted Text) files contain the plain text extracted from the web pages crawled by Common Crawl. They are a subset of the WARC (Web ARChive) files and are useful for text-based analysis without needing to process the original HTML.

Each WET file is a concatenation of records, primarily of the `conversion` type. A `conversion` record typically contains:

*   **WARC-Target-URI**: The URL of the original web page.
*   **WARC-Date**: The date and time the page was crawled.
*   **WARC-Record-ID**: A unique identifier for the record.
*   **Content-Type**: Usually `text/plain`.
*   **Content-Length**: The length of the text content.
*   **The text content**: The extracted text from the web page's body.

The code in the following cells demonstrates how to access and read these records from a WET file.

In [ ]:
# Make sure to run the previous cell (aiIDWwyw0W1q) first to get the first_wet_url

if 'first_wet_url' in locals():
    wet_file_url = first_wet_url
    num_records_to_read = 5 # Define how many records you want to read from the WET file

    print(f"Attempting to read the first {num_records_to_read} records from: {wet_file_url}")

    try:
        response = requests.get(wet_file_url, stream=True)
        response.raise_for_status() # Raise an HTTPError for bad responses

        # Wrap the raw response stream in a BytesIO object
        with io.BytesIO(response.content) as wet_stream:
            records_read = 0
            for record in ArchiveIterator(wet_stream):
                # WET files contain 'conversion' records with extracted text
                if record.rec_type == 'conversion':
                    print(f"\n--- Record {records_read + 1} ---")
                    print(f"URL: {record.rec_headers.get_header('WARC-Target-URI')}")
                    # You can access the text content with record.content_stream().read()
                    # print(f"Content snippet: {record.content_stream().read(200).decode('utf-8', errors='ignore')}...") # Uncomment to see content
                    records_read += 1
                    if records_read >= num_records_to_read:
                        break

        print(f"\nFinished reading {records_read} records.")

    except requests.exceptions.RequestException as e:
        print(f"Error downloading or reading WET file: {e}")
else:
    print("Please run the previous cell first to get the 'first_wet_url'.")

Attempting to read the first 5 records from: https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-43/segments/1759648357851.76/wet/CC-MAIN-20251005114239-20251005144239-00000.warc.wet.gz

--- Record 1 ---
URL: http://093777.vip/WAF/VERIFY/CAPTCHA?info=7QiDJuumWgy3bg3gN5bTNoGRZCaiklsdecGRd8n55T04%2F7hEwsjU0Z%2FXnNINzsdFe%2FQPQ0hG6nA4SnjSsg0%2B6cxGCv0kdjI%3D&from=%2F

--- Record 2 ---
URL: http://123flowers.net/bbs/login.php?url=%2Fshop%2Fsearch.php%3Fq%3DDuy%25C3%25AAn%2BD%25C3%25A1ng

--- Record 3 ---
URL: http://1314buys.com/Products-35441880.html

--- Record 4 ---
URL: http://150.hammerfyr.dk/index.php/godske-wielandt/

--- Record 5 ---
URL: http://1673820.ktem57.com/?PUT=a_show&AID=288112&FID=1673820&R2=&CHANNEL=

Finished reading 5 records.


In [ ]:
# Make sure to run the cell that generates 'wet_paths' (aiIDWwyw0W1q) first

if 'wet_paths' in locals() and wet_paths:
    base = "https://data.commoncrawl.org/"
    num_wet_files_to_process = 1 # Process the first X WET files
    ethnic_groups_found_in_files = {}

    print(f"Processing the first {num_wet_files_to_process} WET files to search for ethnic groups...")

    for i, wet_path in enumerate(wet_paths[:num_wet_files_to_process]):
        wet_file_url = base + wet_path
        print(f"\nProcessing file {i+1}/{num_wet_files_to_process}: {wet_file_url}")

        try:
            response = requests.get(wet_file_url, stream=True)
            response.raise_for_status() # Raise an HTTPError for bad responses

            with io.BytesIO(response.content) as wet_stream:
                for record in ArchiveIterator(wet_stream):
                    if record.rec_type == 'conversion':
                        # Get the text content
                        text_content = record.content_stream().read().decode('utf-8', errors='ignore')

                        # Check if any ethnic group is mentioned in the text content
                        for ethnic_group in ethnic_group_list:
                            if ethnic_group.lower() in text_content.lower():
                                if ethnic_group not in ethnic_groups_found_in_files:
                                    ethnic_groups_found_in_files[ethnic_group] = []
                                # Store the URL and the record's target URI where the group was found
                                ethnic_groups_found_in_files[ethnic_group].append({
                                    'wet_file': wet_file_url,
                                    'target_uri': record.rec_headers.get_header('WARC-Target-URI')
                                })
                                # Optional: print when a group is found
                                # print(f"  Found '{ethnic_group}' in {record.rec_headers.get_header('WARC-Target-URI')}")


        except requests.exceptions.RequestException as e:
            print(f"Error processing WET file {wet_file_url}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred while processing {wet_file_url}: {e}")

    print("\nFinished processing WET files.")

    # Display the results
    print("\nSummary of ethnic groups found and where they were mentioned:")
    if ethnic_groups_found_in_files:
        for ethnic_group, mentions in ethnic_groups_found_in_files.items():
            print(f"\n'{ethnic_group}' found in {len(mentions)} records:")
            # Print up to 5 example locations
            for j, mention in enumerate(mentions[:5]):
                print(f"  - {mention['target_uri']} (from {mention['wet_file']})")
            if len(mentions) > 5:
                print(f"  ... and {len(mentions) - 5} more locations.")
    else:
        print("No ethnic groups were found in the processed WET files.")

else:
    print("Please run the cell that generates the 'wet_paths' list (aiIDWwyw0W1q) first.")

Processing the first 1 WET files to search for ethnic groups...

Processing file 1/1: https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-43/segments/1759648357851.76/wet/CC-MAIN-20251005114239-20251005144239-00000.warc.wet.gz

Finished processing WET files.

Summary of ethnic groups found and where they were mentioned:

'Japanese' found in 537 records:
  - http://1314buys.com/Products-35441880.html (from https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-43/segments/1759648357851.76/wet/CC-MAIN-20251005114239-20251005144239-00000.warc.wet.gz)
  - http://2ard.com/Products-3821059.html (from https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-43/segments/1759648357851.76/wet/CC-MAIN-20251005114239-20251005144239-00000.warc.wet.gz)
  - http://555qi.com/chanpin/lvliao/shuzhi/61.html (from https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-43/segments/1759648357851.76/wet/CC-MAIN-20251005114239-20251005144239-00000.warc.wet.gz)
  - http://asiannudistpictures.com/gallery/Ups_again/?22

next step comprise all asian occurances into one file

# Baseline Stat Analysis

This is just to make sure all ethnicities have some represenation in the data